In [1]:
import numpy as np
import pandas as pd 
import os 
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)

In [6]:
try:
    os.mkdir("team_league_data")
    
except FileExistsError:
    
    print("File Exists")
    pass
    

File Exists


In [12]:
def past_data(data_set):
    
    data_features= data_set.columns
    
    #data set which only posses intiger variables 
    data = data_set.loc[ : ,data_features[2] : data_features[-1] ]
    
    #data  set which posses other than intiger variables.
    data_non_digit = data_set.loc[ : , : data_features[1] ]
    shape = data.shape
    
    #creating copy of "data" in order to set proper data set in same dimention 
    past_data= data.copy()
    
    #iterates throught columns of data which posses numerical values
    for cols in data.columns:
    
        data_col=[]
        #each column will be updates accoring to logic given in error handling blocks
        for i in range(shape[0]):
        
            try:
                
                column = data[cols][i-1] + data[cols][i-2] + data[cols][i-3]
            except KeyError:
                
                column = data[cols][i]
        
            data_col.append(column)
        
        past_data[cols] = data_col

    return pd.concat([data_non_digit , past_data], axis = 1) 
    

In [15]:
def data_producer(data_path,league_name):
    
    try:
        data = pd.read_csv(data_path)
    except FileNotFoundError:
        print("Given Data Set is Wrong")
        
    file_direction="team_league_data/{}".format(league_name)
        
    try:
        os.mkdir(file_direction)
        
    except FileExistsError:
        print("File Exists")
        pass
    
    #slicing relevant data features from "data"
    data_match_related= data.loc[:,"Date":"AR"]
    
    dummy_result=pd.get_dummies(data_match_related["FTR"]) 
    data_match_related = pd.concat( [ data_match_related,dummy_result ], axis = 1).drop([ "FTR" ], axis = 1 )
    
    
    #choosing home match features for teams data
    home_team_columns = ['Date', 'HomeTeam' ,'FTHG','FTAG', 'H','D', 'A' ,'HTHG','HTAG' , 
                         'HS' , 'HST' ,'HF' ,'HC' , 'HY' , 'HR']
    
    
    #choosing away match features for teams data
    away_team_columns = ['Date','AwayTeam' ,'FTAG','FTHG' ,'A' ,'D' ,'H' ,'HTAG','HTHG' , 
                         'AS'  ,'AST' ,'AF' ,'AC' ,'AY' ,'AR']
    
    #detecting team names which played for that season
    team_names = data_match_related["HomeTeam"].unique()
    
    #creating home team match data 
    home_match_data= data_match_related[home_team_columns].rename(columns={"A" : "L", "H" : "W", "FTAG" : "FTGC"
                                                                           ,"HTAG" : "HTGC", "HomeTeam" : "Team", 
                                                           "FTHG" : "FTG", "HTHG" : "HTG",
                                                           "HS" : "S", "HST" : "ST", "HF" : "F",
                                                           "HC" : "C", "HY" : "Y", "HR" : "R"})
   
    #creating away team match data 
    away_match_data= data_match_related[away_team_columns].rename(columns={"A" : "W", "H" : "L", "FTHG" : "FTGC", 
                                                                           "HTHG" : "HTGC", "AwayTeam" : "Team", 
                                                           "FTAG":"FTG", "HTAG" : "HTG",
                                                           "AS" : "S", "AST" : "ST", "AF": "F",
                                                           "AC" : "C", "AY" : "Y", "AR" : "R" })
    
    for team in team_names:
        
        #choosing home match index for the team 
        home_team_index = home_match_data["Team"][lambda x: x == team ].index
        home_team_data = home_match_data.loc[home_team_index]

        #choosing away match index for the team 
        away_team_index = away_match_data["Team"][lambda x: x == team ].index
        away_team_data = away_match_data.loc[away_team_index]
        
        data_2021 = pd.concat([home_team_data, away_team_data], axis=0)
        
        #sorting according to index of original data_set
        team_data = data_2021.sort_index()
        team_data = team_data.reset_index(drop = True)
  
        #creating past 3 match data for each match 
        team_datapast = past_data(team_data)
        
        
        team_datapast.to_csv("{}/{}.csv ".format(file_direction, team), index=False)
        
        
        print("Process finished succesfuly")
    


In [16]:
#creating team based data sets from leagues
direction = 'football_data_uk'
leagues = os.listdir(direction)
for league in leagues :
    data_producer( "{}/{}".format( direction,league ), league[:-4] )

File Exists
Process finished succesfuly
Process finished succesfuly
Process finished succesfuly
Process finished succesfuly
Process finished succesfuly
Process finished succesfuly
Process finished succesfuly
Process finished succesfuly
Process finished succesfuly
Process finished succesfuly
Process finished succesfuly
Process finished succesfuly
Process finished succesfuly
Process finished succesfuly
Process finished succesfuly
Process finished succesfuly
Process finished succesfuly
Process finished succesfuly
File Exists
Process finished succesfuly
Process finished succesfuly
Process finished succesfuly
Process finished succesfuly
Process finished succesfuly
Process finished succesfuly
Process finished succesfuly
Process finished succesfuly
Process finished succesfuly
Process finished succesfuly
Process finished succesfuly
Process finished succesfuly
Process finished succesfuly
Process finished succesfuly
Process finished succesfuly
Process finished succesfuly
Process finished succesf

In [17]:
sample = pd.read_csv( "team_league_data/france/Nantes.csv" )

In [18]:
sample

,Date,Team,FTG,FTGC,W,D,L,HTG,HTGC,S,ST,F,C,Y,R
0,21/08/2020,Nantes,0,0,0,1,0,0,0,6,1,15,3,3,0
1,30/08/2020,Nantes,2,1,1,0,0,2,0,8,5,16,4,3,2
2,13/09/2020,Nantes,1,2,0,0,1,0,1,7,2,11,4,1,0
3,20/09/2020,Nantes,3,3,1,1,1,2,1,21,8,42,11,7,2
4,25/09/2020,Nantes,5,5,1,1,1,2,2,30,10,39,12,6,2
5,03/10/2020,Nantes,3,6,0,1,2,0,3,34,11,35,15,5,0
6,18/10/2020,Nantes,3,6,0,1,2,1,3,38,10,32,13,5,1
7,31/10/2020,Nantes,4,5,1,0,2,3,2,35,15,35,11,6,1
8,08/11/2020,Nantes,4,6,1,0,2,3,1,29,10,36,6,6,1
9,22/11/2020,Nantes,5,4,2,0,1,2,0,34,13,37,9,6,0
